In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
import multiprocessing

In [2]:
traintest = pd.read_pickle('./traintest.p')

train = traintest[traintest.date < '2015-11-01'].drop(['sales','item_cnt_day','ID'], axis=1)
train['year'] = train.date.dt.year
train['month'] = train.date.dt.month
train['day'] = train.date.dt.day
train.drop('date',axis=1, inplace=True)
target = traintest[traintest.date < '2015-11-01']['item_cnt_day']
test = traintest[traintest.date >= '2015-11-01'].drop(['sales','item_cnt_day'],axis=1)
ID = test.ID
test.drop('ID', axis=1, inplace=True)
test['year'] = test.date.dt.year
test['month'] = test.date.dt.month
test['day'] = test.date.dt.day
test.drop('date',axis=1, inplace=True)

In [7]:
xgb_model = xgb.XGBRegressor(n_jobs=multiprocessing.cpu_count() // 2)
#xgb_model = xgb.XGBRegressor(tree_method = 'gpu_hist', objective = 'reg:squarederror')
clf = GridSearchCV(xgb_model, {'max_depth': [2,4,6],
                                'n_estimators': [5,10,25]}, verbose=1,
                                
                    n_jobs=2)
clf.fit(train, target)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


KeyboardInterrupt: 

In [ ]:
print(clf.best_params_)
print(clf.best_score_)

{'max_depth': 2, 'n_estimators': 50}
-0.03945919470759522


In [ ]:
test['item_cnt_day'] = clf.predict(test)
test['ID'] = ID

In [ ]:
submission = test.groupby(['ID']).sum().reset_index().sort_values('ID')[['ID','item_cnt_day']].astype({'ID':'int','item_cnt_day':'float64'})
submission.rename({'item_cnt_day': 'item_cnt_month'}, inplace=True, axis=1)
submission.to_csv('submission.csv', index= False)
submission

,ID,item_cnt_month
0,0,32.880676
1,1,2638.645752
2,2,33.046658
3,3,32.553474
4,4,2664.508301
...,...,...
214195,214195,32.102024
214196,214196,2690.069092
214197,214197,30.992195
214198,214198,2689.664795


In [8]:
import subprocess
subprocess.call('kaggle competitions submit -c competitive-data-science-predict-future-sales -f submission.csv -m "Message"')

FileNotFoundError: [Errno 2] No such file or directory: 'kaggle competitions submit -c competitive-data-science-predict-future-sales -f submission1.csv -m "Message"'